# PATHOverview file lister
This notebook lists all the ndpi files in a given directory and all subdirectories and creates the required Excel file for addition of slide details.

To use:
1. Change the master_dir to the folder you want to list from.
2. Change output_dir to the folder you want your PATHOverview output images generated to. This folder must already exist. An Excel file will be created in this folder with PATHOverview parameters.
3. Run all cells
4. Edit the generated Excel file as needed then use it as the input for [pathoverview_interactive_example.ipynb](pathoverview_interactive_example.ipynb)


The Excel file generated will contain the following (* = required):
- Worksheet 'slides'
  - root*: folder containing the slide file
  - file*: filename for the slide file
  - label: label to be placed on figure output
  - page*: page number to put this slide on
  - order*: position of this slide on page (starting from 0)
  - rotation: rotation of this slide image in degrees
  - mirror: True/1 or False/0 for flipping the image
  - zoom_point: tuple; Point on the slide to zoom in on. Relative to width/height of the slide.
  - crop: tuple; ((tuple of point to centre crop image on), width, height). Relative to width/height of the slide.
  - fig_type:
    - None: overview (cropped and rotated) image with inset zoom
    - 'inverted': zoom with inset overview (cropped and rotated)
    - 'raw': raw overview of the slide (not cropped or rotated)
    - 'slide': raw with an image of the slide label
  - panel_size: tuple; (with, height) of the output image in pixels
  - add_inset: True/1 or False/0 to add an inset image
  - inset_size: tuple; (width, height) of the inset image in pixels
  - zoom_real_size: width in um of the zoom image
  - scale_bar: width in um of the main image scale bar
  - inset_scale_bar: width in um of the inset image scale bar
  - sb_label: True/1 or False/0; add a size label to the scale bars
- Worksheet 'pages'
  - page*: number for this page (to map slides to)
  - title1: title string. This will be evaluated with \**globals at time of use
  - title2: title 2nd line
  - title3: title 3rd line
  - footer: footer string. Evaluated as title
  - filename*: filename with path for output file
  - figsize: tuple; size of the page in inches (for compatability to matplotlib)
  - n_x: number of images wide per page
  - n_y: number of images high per page

In [1]:
# os package handles the file listing
import os

# Pandas handles excel sheet <-> dataframe data handling
import pandas as pd
import numpy as np

# The python builtin Path is used to handle filepaths nicely between windows and mac
from pathlib import Path

# Import pathofigure to use default layout variables to set up pages
from pathoverview import pathofigure

In [2]:
# Path to the directory from which ndpi files will be listed
master_dir = Path(r"./test_files")

# Path to the directory to which files will be output. PATHOverview will fail if this does not exist.
output_dir = Path(r"./test_files")

# Name / path to the Excel file for output of the required dataframes
excel_output = Path(output_dir,"slide_lister_output.xlsx")

In [3]:
# Any files with the strings listed below in their filename will be ignored
exclude_contains = ["EXTRA", "EXCLUDE", "duplicates", "controls"]

# Default figure layout to assign slides to pages automatically
# For example, if this notebook is run on a dir containing 
# 100 images 5 pages of 4x6 images will be specified
n_x = pathofigure.fig_defaults["n_x"] # default = 4
n_y = pathofigure.fig_defaults["n_y"] # default = 6

# Overwrite figure layout
# n_x = 1
# n_y = 1

In [4]:
# A blank list to contain the files found, this is converted into a dataframe below
ndpi_list = []

# First walk through directories / folders
for root, dirs, files in os.walk(Path(master_dir)):

    # if root doesn't contain one of the excluded strings...
    if not any(excl in root for excl in exclude_contains):
    
        # Iterate through each file
        for file in files:
            
            # check if it is an ndpi file but not an annotation file
            if ".ndpi" in file and ".ndpa" not in file:
                
                # if file doesn't contain one of the excluded strings...
                if not any(excl in file for excl in exclude_contains):
                    
                    # add it to the ndpi_list
                    ndpi_list.append((root,file))

# How many files were found?
len(ndpi_list)

2

In [5]:
# Make a dataframe from the ndpi_list
# this contains the filename and the directory for each file
slides_df = pd.DataFrame(ndpi_list,columns=["root","file"])

# Show this df
slides_df

,root,file
0,test_files,OS-1.ndpi
1,test_files,OS-2.ndpi


In [6]:
# Sort the list by filename
slides_df = slides_df.sort_values(["file"]).reset_index(drop= True)
slides_df

,root,file
0,test_files,OS-1.ndpi
1,test_files,OS-2.ndpi


In [11]:
# Add the required columns (as blanks)
slides_df[["label","page","order","rotation","mirror","zoom_point","crop","panel_size",
          "fig_type","add_inset","inset_size","zoom_real_size","scale_bar","inset_scale_bar","sb_label"
          ]] = None

# add links to Excel file for easy navigation to ndpi file
slides_df["rel_path"] = slides_df.apply(lambda L: Path(os.path.relpath(Path(L.root, L.file), excel_output.parent)).as_posix(), axis=1)
slides_df["link"] = '=HYPERLINK("' + slides_df['rel_path'] + '", "Link")'

# assign pages and order for a filled default layout
max_panels = n_x * n_y
# use integer division to assign 'page' number to slides in order
slides_df['page'] = slides_df.groupby(np.arange(len(slides_df))//max_panels).ngroup()
# assign position of slides using cumulative count on each page group
slides_df['order'] = slides_df.groupby("page").cumcount()

slides_df

,root,file,label,page,order,rotation,mirror,zoom_point,crop,panel_size,fig_type,add_inset,inset_size,zoom_real_size,scale_bar,inset_scale_bar,sb_label,rel_path,link
0,test_files,OS-1.ndpi,None,0,0,None,None,None,None,None,None,None,None,None,None,None,None,OS-1.ndpi,"=HYPERLINK(""OS-1.ndpi"", ""Link"")"
1,test_files,OS-2.ndpi,None,0,1,None,None,None,None,None,None,None,None,None,None,None,None,OS-2.ndpi,"=HYPERLINK(""OS-2.ndpi"", ""Link"")"


In [13]:
# write slides_df to the excel output, creating the excel file is it does not exist
if os.path.exists(excel_output):
    with pd.ExcelWriter(excel_output, mode="a", engine="openpyxl", if_sheet_exists="replace") as writer:
        slides_df.to_excel(writer, sheet_name="slides") 
else:
    with pd.ExcelWriter(excel_output, engine="openpyxl") as writer:
        slides_df.to_excel(writer, sheet_name="slides")

### Add a pages worksheet

In [14]:
# Create empty df
pages_df = pd.DataFrame()

# Add required columns
pages_df[["page","title1","title2","title3","footer","filename","figsize","n_x","n_y"]] = None

# Add an entry for all pages used in slides_df
pages_df["page"] = slides_df["page"].unique()

# Specify the footer to include page number
pages_df["footer"] = "Page " + pages_df["page"].astype(str) + ". Generated by PATHOverview."

# Generate the output filename
# Using '+' allows pandas to work at the df level. 
# The path is formatted by PathOVERVIEW at use so here it is system independent 
pages_df["filename"] = str(output_dir) + "\page" + pages_df["page"].astype(str) + ".png"

pages_df["n_x"] = n_x
pages_df["n_y"] = n_y

# Show the df below
pages_df

,page,title1,title2,title3,footer,filename,figsize,n_x,n_y
0,0,NaN,NaN,NaN,Page 0. Generated by PATHOverview.,test_files\page0.png,NaN,4,6


In [15]:
# write pages_df to the excel output, creating the excel file is it does not exist
if os.path.exists(excel_output):
    with pd.ExcelWriter(excel_output, mode="a", engine="openpyxl", if_sheet_exists="replace") as writer:
        pages_df.to_excel(writer, sheet_name="pages") 
else:
    with pd.ExcelWriter(excel_output, engine="openpyxl") as writer:
        pages_df.to_excel(writer, sheet_name="pages")